In [289]:
import pandas as pd
import string
import numpy as np
import seaborn as sns
import os 
from tidypython import *

In [290]:
df = pd.read_csv (r'/Users/aceinna_rishit/Documents/AWS/naukri_jobs.csv')

In [291]:
#creating job_title table
job_posting = pd.DataFrame()
job_posting['job_title'] = df['Title'].str.strip()
job_posting['company_name'] = df['Company_Name'].str.strip()
job_posting['category'] = df['Category'].str.strip()
job_posting['role_category'] = df['Role_Category'].str.strip()
job_posting['job_role'] = df['Job_Role'].str.strip()
job_posting['functional_area'] = df['Functional_Area'].str.strip()
job_posting['employment_type'] = df['Employment_Type'].str.strip()
job_posting['salary'] = df['Salary'].str.strip()
job_posting['experience'] = df['Experience'].str.strip()
job_posting['location'] = df['Location'].str.strip()
job_posting['skills'] = df['Extracted_Skills'].str.strip()
job_posting['job_url'] = df['Url'].str.strip()
job_posting.insert(0,'job_id', range(0,len(job_posting)))

In [292]:
# WARNING: Removes the entry if any of the groupby field is empty
grouped_title = job_posting.groupby(['job_title','company_name', 'category', 'role_category','functional_area','employment_type','salary','experience','location','skills','job_url'])
job_title = pd.DataFrame()
job_title = grouped_title.agg({'job_id':'size'}).rename(columns={'job_id':'count_jobs'}).reset_index()
job_title.insert(0,'title_id', range(0,len(job_title['job_title'])))
# job_title.set_index('title_id')


In [293]:
#  Create a DF of unique categories from job_posting['company_name']
company_unique = pd.DataFrame(data=job_posting['company_name'].unique(), columns=["company_name"])
# Add inddex Column
company_unique.insert(0,'company_id', range(0,len(job_title['company_name'])-1))
company_dict = dict(zip(company_unique['company_name'], company_unique['company_id']))


In [294]:
#  Create a DF of unique categories from job_posting['category']
category_unique = pd.DataFrame(data=job_posting['category'].unique(), columns=["category"])
# Add inddex Column
category_unique.reset_index(level=0, inplace=True)
# Here .insert() to insert index column gives indexing error so manually changing the name of column after creation
category_unique.rename(columns={'index': 'caterogy_id'}, inplace=True)
category_dict = dict(zip(category_unique['category'], category_unique['caterogy_id']))


In [295]:
#  Create a DF of unique categories from job_posting['role_category']
role_category_unique = pd.DataFrame(data=job_posting['role_category'].dropna().unique(), columns=["role_category"])
# Add inddex Column
role_category_unique.reset_index(level=0, inplace=True)
# Here .insert() to insert index column gives indexing error so manually changing the name of column after creation
role_category_unique.rename(columns={'index': 'role_category_id'}, inplace=True)
role_category_dict = dict(zip(role_category_unique['role_category'], role_category_unique['role_category_id']))


In [296]:
#  Create a DF of unique categories from job_posting['category']
job_role_unique = pd.DataFrame(data=job_posting['job_role'].unique(), columns=["job_role"])
# Add inddex Column
job_role_unique.reset_index(level=0, inplace=True)
# Here .insert() to inser index column give indexing error so manually changing the name of column after creation
job_role_unique.rename(columns={'index': 'job_role_id'}, inplace=True)
job_role_dict = dict(zip(job_role_unique['job_role'], job_role_unique['job_role_id']))


In [297]:
#  Create a DF of unique categories from job_posting['category']
employment_data_unique = pd.DataFrame(job_title.employment_type.str.split(',').tolist(), index=job_title.title_id, columns = ['employment_type','employment_duration'])
employment_type_unique = pd.DataFrame(data=employment_data_unique['employment_type'].unique(), columns=["employment_type"])
employment_type_unique.reset_index(level=0, inplace=True)
employment_type_unique.rename(columns={'index': 'employment_type_id'}, inplace=True)
employment_type_dict = dict(zip(employment_type_unique['employment_type'], employment_type_unique['employment_type_id']))

employment_duration_unique = pd.DataFrame(data=employment_data_unique['employment_duration'].unique(), columns=["employment_duration"])
employment_duration_unique.reset_index(level=0, inplace=True)
employment_duration_unique.rename(columns={'index': 'employment_duration_id'}, inplace=True)
employment_duration_unique = dict(zip(employment_duration_unique['employment_duration'], employment_duration_unique['employment_duration_id']))

In [298]:
#  Create a DF of unique categories from job_posting['skills']
posting_skills = pd.DataFrame(job_title.skills.str.split(',').tolist(), index=job_title.title_id).stack()
posting_skills = posting_skills.str.lower()
posting_skills = posting_skills.str.strip()
posting_skills = posting_skills.reset_index([0, 'title_id'])
posting_skills.columns = ['title_id', 'skills']
# posting_skills.to_csv(r'/Users/aceinna_rishit/Documents/AWS/posting_skills_temp.csv', index = False)
skill_table = pd.DataFrame()
skill_table = posting_skills.groupby("skills").agg({"title_id": "nunique"}).rename(columns={'title_id': 'count_jobs'}).reset_index([0, 'skills'])
skill_table.insert(0,'skill_id', range(0,len(skill_table['skills'])))
skill_dict = dict(zip(skill_table['skills'], skill_table['skill_id']))

# Replece skills with skill_id in posting_skills['skills']
posting_skills['skills'] = posting_skills['skills'].replace(skill_dict)
# Update posting_skills['skills'] to posting_skills['skills_id']
posting_skills.rename(columns={'skills': 'skills_id'}, inplace=True)


In [299]:
# Treat Functional Area field same as skills, create a unique table for funcitonal area and map functional area with the job_posting
#  Create a DF of unique categories from job_posting['functional_area']
posting_functional_areas = pd.DataFrame(job_title.functional_area.str.split(', |/').tolist(), index=job_title.title_id).stack()
posting_functional_areas = posting_functional_areas.str.lower()
posting_functional_areas = posting_functional_areas.str.strip()
posting_functional_areas = posting_functional_areas.reset_index([0, 'title_id'])
posting_functional_areas.columns = ['title_id', 'functional_area']
# posting_functional_areas.to_csv(r'/Users/aceinna_rishit/Documents/AWS/posting_functional_areas_temp.csv', index = False)

functional_area_table = pd.DataFrame()
functional_area_table = posting_functional_areas.groupby("functional_area").agg({"title_id": "nunique"}).rename(columns={'title_id': 'count_jobs'}).reset_index([0, 'functional_area'])
functional_area_table.insert(0,'functional_area_id', range(0,len(functional_area_table['functional_area'])))
functional_area_dict = dict(zip(functional_area_table['functional_area'], functional_area_table['functional_area_id']))

# Replece functional_area with functional_area_id in posting_functional_area['functional_area']
posting_functional_areas['functional_area'] = posting_functional_areas['functional_area'].replace(functional_area_dict)
# Update posting_functional_area['functional_area'] to posting_functional_area['functional_area_id']
posting_functional_areas.rename(columns={'functional_area': 'functional_area_id'}, inplace=True)


In [300]:
job_posting.insert(2, "company_id", job_posting["company_name"].map(company_dict))
job_posting.insert(4, "category_id", job_posting["category"].map(category_dict))
job_posting.insert(6, "role_category_id", job_posting["role_category"].map(role_category_dict))
job_posting.insert(8, "job_role_id", job_posting["job_role"].map(job_role_dict))
job_posting.drop('functional_area', inplace=True, axis=1)

# Process Salary Column
job_posting['salary'] = job_posting['salary'].map(lambda x: x.rstrip('PA.'))
job_posting['salary'] = job_posting['salary'].map(lambda x: x.replace(',',''))
job_posting[['salary_min','salary_max']] = job_posting.salary.str.split('-',expand=True)
job_posting.salary_min.replace('Not disclosed', np.nan, regex=True, inplace=True)
job_posting.salary_max.fillna(value=np.nan,inplace=True)
job_posting.drop('salary', inplace=True, axis=1)

# Process Experience Range
job_posting['experience'] = job_posting['experience'].str.rstrip(' Yrs')
job_posting[['experience_min','experience_max']] = job_posting.experience.str.split('-',expand=True)
job_posting.experience_min.fillna(value=np.nan,inplace=True)
job_posting.experience_max.fillna(value=np.nan,inplace=True)
job_posting.drop('experience', inplace=True, axis=1)

#  Process Location
# print(job_posting['location'].count)
cont = job_posting['location'].count()
print(cont)
print(job_posting.count())

job_posting.location.fillna(value=np.nan,inplace=True)
job_posting[['location','location_comments']] = job_posting.location.str.split(',',expand=True)
job_posting.location_comments.fillna(value=np.nan,inplace=True)
job_posting.drop('location', inplace=True, axis=1)

print(job_posting['location'].head())
print(job_posting['location_comments'].head())


21
job_id              21
job_title           21
company_id          21
company_name        21
category_id         21
category            21
role_category_id    20
role_category       20
job_role_id         21
job_role            21
employment_type     21
location            21
skills              21
job_url             21
salary_min          20
salary_max          20
experience_min      20
experience_max      20
dtype: int64


ValueError: Columns must be same length as key

In [ ]:
job_title.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/job_title.csv', index = False)
company_unique.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/company_unique.csv', index = False)
category_unique.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/category_unique.csv', index = False)
role_category_unique.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/role_category_unique.csv', index = False)
job_role_unique.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/job_role_unique.csv', index = False)
skill_table.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/skills_table.csv', index = False)
posting_skills.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/posting_skills.csv', index = False)
functional_area_table.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/functional_area_table.csv', index = False)
posting_functional_areas.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/posting_functional_areas.csv', index = False)
job_posting.to_csv(r'/Users/aceinna_rishit/Documents/AWS/tables/job_posting.csv', index = False)